In [22]:
# !pip install psycopg2

In [168]:
import psycopg2
import numpy as np
from scipy.special import rel_entr
from scipy.special import kl_div
from scipy.stats import entropy
from collections import defaultdict
# Making connection to postgres database
conn = psycopg2.connect(database='SeeDB', user='postgres', password='password')

cur = conn.cursor()
cur.execute('SELECT * FROM census1 LIMIT 5')
head_5 = cur.fetchall()

print("First 5 rows of census table:")
for row in head_5:
    print(row)

cur.execute('SELECT COUNT(*) FROM census1')
total_rows = cur.fetchone()[0]
print("Total rows in census table:", total_rows)

# conn.close()


First 5 rows of census table:
(1, 39, ' State-gov', 77516, ' Bachelors', 13, ' Never-married', ' Adm-clerical', ' Not-in-family', ' White', ' Male', 2174, 0, 40, ' United-States', ' <=50K')
(2, 50, ' Self-emp-not-inc', 83311, ' Bachelors', 13, ' Married-civ-spouse', ' Exec-managerial', ' Husband', ' White', ' Male', 0, 0, 13, ' United-States', ' <=50K')
(3, 38, ' Private', 215646, ' HS-grad', 9, ' Divorced', ' Handlers-cleaners', ' Not-in-family', ' White', ' Male', 0, 0, 40, ' United-States', ' <=50K')
(4, 53, ' Private', 234721, ' 11th', 7, ' Married-civ-spouse', ' Handlers-cleaners', ' Husband', ' Black', ' Male', 0, 0, 40, ' United-States', ' <=50K')
(5, 28, ' Private', 338409, ' Bachelors', 13, ' Married-civ-spouse', ' Prof-specialty', ' Wife', ' Black', ' Female', 0, 0, 40, ' Cuba', ' <=50K')
Total rows in census table: 32561


In [169]:
def kl_divergence(prob1, prob2):
    prob1 = prob1/np.sum(np.asarray(prob1, dtype=float))
    prob2 = prob2/np.sum(np.asarray(prob2, dtype=float))
    # Clipping values of prob2 between eps and positive infinity to avoid dividing by 0
    prob2 = np.clip(prob2, np.finfo(float).eps, None)
    return np.sum(prob1 * np.log(prob1 / prob2))
    
# # Testing kl divergence
# prob1 = [0.25, 0.33, 0.23, 0.19]
# prob2 = [0.21, 0.21, 0.32, 0.26]
# kl_divergence(prob1, prob2)

In [170]:
query_ds_cond = "marital_status in (' Married-civ-spouse', ' Married-spouse-absent', ' Married-AF-spouse')"
ref_ds_cond = "marital_status in (' Divorced', ' Never-married', ' Separated', ' Widowed')"

measures = ['age', 'capital_gain', 'capital_loss', 'hours_per_week']
dimensions = ['workclass', 'education', 'occupation', 'relationship', 'race', 'sex', 'native_country', 'income']
agg_functions = ['avg', 'sum', 'min', 'max', 'count']

In [171]:
views = defaultdict(lambda: defaultdict(set))
for dim in dimensions:
    for measure in measures:
        views[dim][measure] = (set(agg_functions))


In [172]:
def check_or_correct_no_of_rows(attribute, query_dataset_rows, reference_dataset_rows, start_idx, end_idx):
    if len(query_dataset_rows) != len(reference_dataset_rows):
        zeros_row = np.zeros((1, reference_dataset_rows.shape[1]), dtype=float)
        query_dataset_query = f"""
        SELECT __atr__, count(*)
        FROM (
            SELECT DISTINCT({attribute}) AS __atr__
            FROM census
            WHERE {query_ds_cond}
            AND id >= %s AND id < %s
        ) AS attrs
        LEFT OUTER JOIN census ON __atr__ = {attribute}
        WHERE {query_ds_cond}
        GROUP BY __atr__
        ORDER BY __atr__
    """
        cur.execute(query_dataset_query, (start_idx, end_idx))
        query_dataset = np.array(cur.fetchall()).astype(str)

        ref_dataset_query = f"""
        SELECT __atr__, count(*)
        FROM (
            SELECT DISTINCT({attribute}) AS __atr__
            FROM census
            WHERE {ref_ds_cond}
            AND id >= %s AND id < %s
        ) AS attrs
        LEFT OUTER JOIN census ON __atr__ = {attribute}
        WHERE {query_ds_cond}
        GROUP BY __atr__
        ORDER BY __atr__
    """
        cur.execute(ref_dataset_query, (start_idx, end_idx))
        ref_dataset = np.array(cur.fetchall()).astype(str)

        for i in range(len(query_dataset)):
            if (i<len(ref_dataset) and query_dataset[i][0] != ref_dataset[i][0]) or (i>=len(ref_dataset)):
                reference_dataset_rows = np.insert(reference_dataset_rows, i, zeros_row, axis=0)
                ref_dataset = np.insert(ref_dataset, i, np.array([query_dataset[i][0], '0']), axis=0)
    return reference_dataset_rows
    

In [173]:
# Creating partitions of the data
num_partitions = 10
partition_size = total_rows // num_partitions
phase_num = 0
view_num = -1
for i in range(num_partitions):
    start_idx = i * partition_size + 1
    end_idx = 1 + ((i + 1) * partition_size - 1 if i != num_partitions - 1 else total_rows - 1)
    print(start_idx, end_idx)
    phase_num += 1
    view_distance_mappings = dict()
    view_dist = []
    view_num = -1
    for attribute in views:
        selections = ''
        for measure in views[attribute]:
            for func in views[attribute][measure]:
                selections += f'COALESCE({func}({measure}), 0), '
        selections = selections[:-2]          

        query_dataset_query = f"""
        SELECT {selections}
        FROM (
            SELECT DISTINCT({attribute}) AS __atr__
            FROM census
            WHERE {query_ds_cond}
            AND id >= %s AND id < %s
        ) AS attrs
        LEFT OUTER JOIN census ON __atr__ = {attribute}
        WHERE {query_ds_cond}
        GROUP BY __atr__
        ORDER BY __atr__
    """
    
        reference_dataset_query = f"""
        SELECT {selections}
        FROM (
            SELECT DISTINCT({attribute}) AS __atr__
            FROM census
            WHERE {ref_ds_cond}
            AND id >= %s AND id < %s
        ) AS attrs
        LEFT OUTER JOIN census ON __atr__ = {attribute}
        WHERE {ref_ds_cond}
        GROUP BY __atr__
        ORDER BY __atr__
    """
        cur.execute(query_dataset_query, (start_idx, end_idx))
        
            # print(query_dataset_query, (start_idx, end_idx))
        query_dataset_rows = np.array(cur.fetchall()).astype(float)
        print(query_dataset_rows)
        cur.execute(reference_dataset_query, (start_idx, end_idx))
        reference_dataset_rows = np.array(cur.fetchall()).astype(float)
        # print("Dimensions of reference_dataset_rows1:", reference_dataset_rows.shape)
        reference_dataset_rows = check_or_correct_no_of_rows(attribute, query_dataset_rows, reference_dataset_rows, start_idx, end_idx)
        # print("Dimensions of reference_dataset_rows2:", reference_dataset_rows.shape)

        # Pruning based optimization
        column_num = -1
        for measure in views[attribute]:
            for func in views[attribute][measure]:
                view_num += 1
                column_num += 1
                # d = 0
                try:
                    d = kl_divergence(query_dataset_rows[:, column_num], reference_dataset_rows[:, column_num])
                    print("d is: ", d)
                except Exception as e:
                    # print("Attribute name is: ", attribute)
                    pass
                if np.isnan(d):
                    # print(attribute, measure, func)
                    view_dist.append(0)
                else:
                    view_dist.append(d)
                view_distance_mappings[view_num] = (attribute, measure, func)
                
    pruned_candidates = []
    pruned_candidates_indexes = []
    index = []
    # print("View_dist are: ", view_dist)
    kl_divg = np.array(view_dist)
    if phase_num != 1:
        num_candidates = len(kl_divg)
        # print("kl_divg is: ", kl_divg)
        # sort the kl divergences
        sorted_divergences = np.sort(kl_divg)[::-1]
        # print("kl_sorted is:", sorted_divergences)
        index = np.argsort(kl_divg)[::-1]
        if phase_num == num_partitions:
            pruned_candidates_indexes = index[5:]
        else:
            # Calculate the confidence interval
            delta = 0.05
            numerator = (1 -(phase_num/num_partitions)) * (2*np.log(np.log(phase_num+1)) + np.log(np.pi**2/(3*delta)))
            denominator = 2*(phase_num+1)
            confidence_interval_error_margin = np.sqrt(numerator/denominator)
            # min_factor = kl_sorted[4] - confidence_interval_error_margin
            for i in range(5, num_candidates):    
                if sorted_divergences[4] - confidence_interval_error_margin > confidence_interval_error_margin + sorted_divergences[i]:
                    pruned_candidates_indexes = index[i:]
    for index_value in pruned_candidates_indexes:
        pruned_candidates.append(view_distance_mappings[index_value])
    # pruned_candidates = [view_distance_mappings[idx]
    #                         for idx in pruned_candidates_indexes]
    for dimension, measure, func in pruned_candidates:
        views[dimension][measure].remove(func)
        if not views[dimension][measure]:
            del views[dimension][measure]
            if not views[dimension]:
                del views[dimension]
                    
recommended_views = []     
final_reccos = [(d, m, f) for d in views for m in views[d] for f in views[d][m]]
print("Recommended views are: ", final_reccos) 


1 3256
[[3.57600000e+04 5.36131934e+01 6.67000000e+02 1.90000000e+01
  9.00000000e+01 7.32558000e+05 1.09828786e+03 6.67000000e+02
  0.00000000e+00 9.99990000e+04 5.31560000e+04 7.96941529e+01
  6.67000000e+02 0.00000000e+00 2.60300000e+03 2.12290000e+04
  3.18275862e+01 6.67000000e+02 1.00000000e+00 9.90000000e+01]
 [2.15400000e+04 4.46887967e+01 4.82000000e+02 1.90000000e+01
  9.00000000e+01 5.18245000e+05 1.07519710e+03 4.82000000e+02
  0.00000000e+00 2.00510000e+04 6.03990000e+04 1.25309129e+02
  4.82000000e+02 0.00000000e+00 2.41500000e+03 2.04930000e+04
  4.25165975e+01 4.82000000e+02 8.00000000e+00 8.00000000e+01]
 [4.60590000e+04 4.40755981e+01 1.04500000e+03 2.00000000e+01
  9.00000000e+01 1.26071300e+06 1.20642392e+03 1.04500000e+03
  0.00000000e+00 9.99990000e+04 1.49696000e+05 1.43249761e+02
  1.04500000e+03 0.00000000e+00 2.41500000e+03 4.42230000e+04
  4.23186603e+01 1.04500000e+03 4.00000000e+00 9.90000000e+01]
 [4.14649000e+05 4.12627127e+01 1.00490000e+04 1.70000000e+0

C:\Users\prana\AppData\Local\Temp\ipykernel_26044\73027522.py:2: RuntimeWarning: invalid value encountered in divide
  prob1 = prob1/np.sum(np.asarray(prob1, dtype=float))
C:\Users\prana\AppData\Local\Temp\ipykernel_26044\73027522.py:3: RuntimeWarning: invalid value encountered in divide
  prob2 = prob2/np.sum(np.asarray(prob2, dtype=float))


[[4.99300000e+03 4.02661290e+01 1.24000000e+02 1.90000000e+01
  7.40000000e+01 8.61760000e+04 6.94967742e+02 1.24000000e+02
  0.00000000e+00 1.50240000e+04 5.36400000e+03 4.32580645e+01
  1.24000000e+02 0.00000000e+00 1.97700000e+03 5.20000000e+03
  4.19354839e+01 1.24000000e+02 8.00000000e+00 7.00000000e+01]
 [2.31980000e+04 4.23321168e+01 5.48000000e+02 1.90000000e+01
  9.00000000e+01 1.10547000e+06 2.01728102e+03 5.48000000e+02
  0.00000000e+00 9.99990000e+04 7.38030000e+04 1.34677007e+02
  5.48000000e+02 0.00000000e+00 2.45700000e+03 2.32550000e+04
  4.24361314e+01 5.48000000e+02 1.00000000e+00 9.90000000e+01]
 [3.82770000e+04 4.25773081e+01 8.99000000e+02 1.80000000e+01
  8.40000000e+01 1.02590300e+06 1.14116018e+03 8.99000000e+02
  0.00000000e+00 9.99990000e+04 7.71220000e+04 8.57864294e+01
  8.99000000e+02 0.00000000e+00 2.37700000e+03 3.67590000e+04
  4.08887653e+01 8.99000000e+02 2.00000000e+00 9.90000000e+01]
 [4.40800000e+03 3.67333333e+01 1.20000000e+02 1.90000000e+01
  7.7

C:\Users\prana\AppData\Local\Temp\ipykernel_26044\73027522.py:6: RuntimeWarning: divide by zero encountered in log
  return np.sum(prob1 * np.log(prob1 / prob2))
C:\Users\prana\AppData\Local\Temp\ipykernel_26044\73027522.py:6: RuntimeWarning: invalid value encountered in multiply
  return np.sum(prob1 * np.log(prob1 / prob2))


[[1.67280000e+04 4.59560440e+01 3.64000000e+02 1.70000000e+01
  9.00000000e+01 3.07561000e+05 8.44947802e+02 3.64000000e+02
  0.00000000e+00 9.99990000e+04 1.89380000e+04 5.20274725e+01
  3.64000000e+02 0.00000000e+00 2.17900000e+03 1.51690000e+04
  4.16730769e+01 3.64000000e+02 1.00000000e+00 9.90000000e+01]
 [1.57130000e+04 4.21260054e+01 3.73000000e+02 1.80000000e+01
  9.00000000e+01 1.29880000e+05 3.48203753e+02 3.73000000e+02
  0.00000000e+00 9.38600000e+03 2.99510000e+04 8.02975871e+01
  3.73000000e+02 0.00000000e+00 2.17900000e+03 1.50930000e+04
  4.04638070e+01 3.73000000e+02 3.00000000e+00 8.80000000e+01]
 [5.71600000e+03 4.17226277e+01 1.37000000e+02 1.90000000e+01
  7.30000000e+01 8.37560000e+04 6.11357664e+02 1.37000000e+02
  0.00000000e+00 1.50240000e+04 5.07800000e+03 3.70656934e+01
  1.37000000e+02 0.00000000e+00 1.84800000e+03 5.83400000e+03
  4.25839416e+01 1.37000000e+02 1.00000000e+01 9.90000000e+01]
 [4.37600000e+03 4.70537634e+01 9.30000000e+01 2.10000000e+01
  8.1